# Workout Data Workbook 
by Alex Guerrero

### Packages Imported

In [1]:
# from email import header
# import streamlit as st
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

### CSV File Import

In [2]:
filename = 'data/main.csv'
workout_data = pd.read_csv(filename)

## Exploring of the data

In [3]:
# Printing the first 5 rows
workout_data.head()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume,Unnamed: 9
0,04/04/2022,Dumbbell Shoulder Press,Shoulders,1.0,10.0,40.0,NaN,NaN,400.0,NaN
1,04/04/2022,Dumbbell Shoulder Press,Shoulders,2.0,10.0,40.0,NaN,NaN,400.0,NaN
2,04/04/2022,Dumbbell Shoulder Press,Shoulders,3.0,10.0,40.0,NaN,NaN,400.0,NaN
3,04/04/2022,Dumbbell Bench Press,Chest,1.0,10.0,30.0,NaN,NaN,300.0,NaN
4,04/04/2022,Dumbbell Bench Press,Chest,2.0,10.0,30.0,NaN,NaN,300.0,NaN


In [4]:
# Printing the last 5 rows
workout_data.tail()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume,Unnamed: 9
11003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [5]:
# date shape
workout_data.shape

(11008, 10)

In [6]:
# printing data columns
workout_data.columns

Index(['Date', 'Exercise', 'Muscle_group', 'Sets', 'Reps', 'Weight',
       'Duration', 'Note', 'Total_volume', 'Unnamed: 9'],
      dtype='object')

In [7]:
# Dara more information
workout_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11008 entries, 0 to 11007
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          2346 non-null   object 
 1   Exercise      2346 non-null   object 
 2   Muscle_group  2346 non-null   object 
 3   Sets          2292 non-null   float64
 4   Reps          2266 non-null   float64
 5   Weight        1635 non-null   float64
 6   Duration      1 non-null      object 
 7   Note          208 non-null    object 
 8   Total_volume  11008 non-null  float64
 9   Unnamed: 9    2 non-null      object 
dtypes: float64(4), object(6)
memory usage: 860.1+ KB


In [8]:
# checking and removing NaN rows
workout_data[workout_data.Date.isnull()]

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume,Unnamed: 9
2346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
11003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


### Dataframe Clean Up

In [9]:
# columns clean up
data_columns = ['Date', 'Exercise', 'Muscle_group', 'Sets', 'Reps', 'Weight', 'Duration', 'Note', 'Total_volume']
workout_data = workout_data[data_columns]

# rows clean up - removing NaN rows
workout_data = workout_data[workout_data.Date.notnull()]

# changing date field to datetime
workout_data['Date'] = pd.to_datetime(workout_data['Date'], format="%m/%d/%Y")

# sorting dataframe according to date, exercise, sets
workout_data = workout_data.sort_values(by=['Date','Exercise','Sets'])

In [10]:
# re-checking dataframe shape
workout_data.shape

(2346, 9)

### Calculations

In [11]:
# Date Parameters -- Calculations
today = date.today()
start_of_week = today - relativedelta(days=today.weekday())
end_of_Week = start_of_week + relativedelta(days=6)
first_day_of_month = today.replace(day=1)
next_month_first_day = today + relativedelta(months=1, day=1)

# Formatting date parameters into string
today = today.strftime('%m-%d-%Y')
start_of_week = start_of_week.strftime('%m-%d-%Y')
end_of_Week = end_of_Week.strftime('%m-%d-%Y')
first_day_of_month = first_day_of_month.strftime('%m-%d-%Y')
next_month_first_day = next_month_first_day.strftime('%m-%d-%Y')

## KPIs

In [12]:
# total count of record in current week
weekly_count = len(workout_data.query(" Date >= @start_of_week & Date <= @end_of_Week")['Date'].unique())
print(f"Weekly Record Count: {weekly_count}")

Weekly Record Count: 1


In [13]:
# total count of record in current month
monthly_count = len(workout_data.query(" Date >= @first_day_of_month & Date < @next_month_first_day")['Date'].unique())
print(f"Montly Record Count: {monthly_count}")

Montly Record Count: 4


In [14]:
# unique date total record count
total_records_count = len(workout_data.Date.unique())
print(f"Total record count: {total_records_count}")

Total record count: 155


In [15]:
# total volume per day
total_volume_per_day = workout_data[["Date","Total_volume"]].groupby(by="Date").sum("Total_volume")
total_volume_per_day.reset_index()

,Date,Total_volume
0,2022-04-04,7070.0
1,2022-04-06,3230.0
2,2022-04-09,1020.0
3,2022-04-11,2550.0
4,2022-04-12,2200.0
...,...,...
150,2023-02-28,9285.0
151,2023-03-02,3620.0
152,2023-03-04,4850.0
153,2023-03-05,9970.0


In [16]:
# bodyweight analysis
BODYWEIGHT_EXERCISES = ["Pull Up", "Chin Up", "Dips", "Push Up", "Squat"]
columns_name = ["Exercise", "Min", "Max", "Avg"]
df = []

for exercise in BODYWEIGHT_EXERCISES:
    min = round(workout_data[workout_data["Exercise"] == exercise].min()["Reps"])
    max = round(workout_data[workout_data["Exercise"] == exercise].max()["Reps"])
    avg = round(workout_data[workout_data["Exercise"] == exercise].mean(numeric_only=True)["Reps"])
    
    entry = [exercise, min, max, avg]
    df.append(entry)
    
df = pd.DataFrame(df, columns=columns_name)
df

<ipython-input-16-c4c4b6324149>:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  min = round(workout_data[workout_data["Exercise"] == exercise].min()["Reps"])
<ipython-input-16-c4c4b6324149>:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  max = round(workout_data[workout_data["Exercise"] == exercise].max()["Reps"])


,Exercise,Min,Max,Avg
0,Pull Up,2,10,5
1,Chin Up,3,10,6
2,Dips,3,20,10
3,Push Up,10,24,12
4,Squat,15,25,21


### Latest Record

In [17]:
latest_record = workout_data[workout_data['Date'] == workout_data['Date'].unique()[-1]]
latest_record

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume
2325,2023-03-07,Barbell Flat Bench Press,Chest,1.0,8.0,65.0,NaN,NaN,520.0
2326,2023-03-07,Barbell Flat Bench Press,Chest,2.0,5.0,85.0,NaN,NaN,425.0
2327,2023-03-07,Barbell Flat Bench Press,Chest,3.0,3.0,135.0,NaN,NaN,405.0
2328,2023-03-07,Barbell Flat Bench Press,Chest,4.0,3.0,135.0,NaN,NaN,405.0
2329,2023-03-07,Barbell Flat Bench Press,Chest,5.0,2.0,135.0,NaN,NaN,270.0
2330,2023-03-07,Dips,Triceps,1.0,10.0,NaN,NaN,NaN,0.0
2331,2023-03-07,Dips,Triceps,2.0,8.0,NaN,NaN,NaN,0.0
2332,2023-03-07,Dips,Triceps,3.0,8.0,NaN,NaN,NaN,0.0
2333,2023-03-07,Dips,Triceps,4.0,8.0,NaN,NaN,NaN,0.0
2339,2023-03-07,Dumbbell Incline Bench Press,Chest,1.0,10.0,35.0,NaN,NaN,350.0


In [18]:
print(f"Recent muscle groups worked out: {workout_data[workout_data['Date'] == workout_data['Date'].unique()[-1]]['Muscle_group'].unique()}")


Recent muscle groups worked out: ['Chest' 'Triceps' 'Shoulders' 'Back']


In [19]:
# what muscle did we work out last
for unique_muscle in latest_record['Muscle_group'].unique():
    print(unique_muscle)

Chest
Triceps
Shoulders
Back
